## Load processed data

In [24]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)

tm = pd.read_csv("../data/processed/arsenal_team_match_stats.csv")
pm = pd.read_csv("../data/processed/arsenal_player_match_stats.csv")
ars_sched = pd.read_csv("../data/processed/arsenal_schedule.csv")

tm.shape, pm.shape, ars_sched.shape

((304, 23), (4352, 36), (304, 17))

- **Pandas:** Core Python library for tabular data (tables, csvs, DataFrames)
- **Matplotlib:** Standard plotting library
- **pd.set_option("display.max_columns", 200):** By default, pandas hides columns if there are many, and since we have many columns we display a DataFrame, show up to 200 columns instead of truncating them
- **pd.read_csv():** Reads a CSV file into a Pandas DataFrame
- **.shape:** Checks dataset size, (rows, columns)

## Basic data sanity checks

In [2]:
tm.isna().sum().sort_values(ascending=False).head(10)

Notes         304
Attendance     44
xG             38
xGA            38
league          0
season          0
team            0
round           0
time            0
date            0
dtype: int64

In [3]:
pm.isna().sum().sort_values(ascending=False).head(10)

Passes_Cmp%            571
Performance_Touches    529
Performance_Tkl        529
Take-Ons_Att           529
Expected_xAG           529
Passes_Cmp             529
SCA_SCA                529
SCA_GCA                529
Carries_Carries        529
Passes_PrgP            529
dtype: int64

## Final join sanity check

In [26]:
# Making sure the dates align
ars_sched["date"] = pd.to_datetime(ars_sched["date"])
tm_clean["date"] = pd.to_datetime(tm_clean["date"])

In [28]:
# Making sure venue labels match
tm_clean["venue"] = tm_clean["venue"].astype(str).str.strip().str.title()


In [29]:
# Creating opponent +GF/GA in the schedule
def parse_score(score):
    s = str(score).replace("–", "-")
    if "-" not in s:
        return (None, None)
    a, b = s.split("-", 1)
    try:
        return int(a.strip()), int(b.strip())
    except:
        return (None, None)

hg_ag = ars_sched["score"].apply(parse_score)
ars_sched["home_goals"] = [x[0] for x in hg_ag]
ars_sched["away_goals"] = [x[1] for x in hg_ag]

ars_sched["venue"] = ars_sched["home_team"].astype(str).str.strip().apply(lambda x: "Home" if x == "Arsenal" else "Away")
ars_sched["opponent"] = ars_sched.apply(lambda r: r["away_team"] if r["venue"] == "Home" else r["home_team"], axis=1)

ars_sched["gf"] = ars_sched.apply(lambda r: r["home_goals"] if r["venue"] == "Home" else r["away_goals"], axis=1)
ars_sched["ga"] = ars_sched.apply(lambda r: r["away_goals"] if r["venue"] == "Home" else r["home_goals"], axis=1)


In [30]:
# Now creating the mapping table
sched_map = ars_sched[["date", "venue", "opponent", "gf", "ga", "game_id"]].drop_duplicates()
sched_map.head()


,date,venue,opponent,gf,ga,game_id
0,2016-08-14,Home,Liverpool,3,4,0e815975
1,2016-08-20,Away,Leicester City,0,0,7dcbb8b3
2,2016-08-27,Away,Watford,3,1,b123da8c
3,2016-09-10,Home,Southampton,2,1,a8053d4d
4,2016-09-17,Away,Hull City,4,1,f2805e6a


In [31]:
# Merging game_id into tm_clean
tm_with_id = tm_clean.merge(
    sched_map,
    left_on=["date", "venue", "opponent", "GF", "GA"],
    right_on=["date", "venue", "opponent", "gf", "ga"],
    how="left"
)

tm_with_id["game_id"].isna().sum(), tm_with_id.shape


(np.int64(0), (304, 28))

In [32]:
# Now joining player rows to match outcomes
player_match_joined = pm.merge(
    tm_with_id[["game_id", "season", "date", "opponent", "venue", "goal_diff", "has_xg"]],
    on="game_id",
    how="left"
)

player_match_joined["goal_diff"].isna().sum()


np.int64(0)

In [ ]:
## Player-level rows are joined to match 